## Overview of the Data.

IRI dataset is a scanner level data set encompasing about 12 years, 93 markets in the United States. Scanner means the level of observation is at the upc-barcode scanner level. 

There are several layers to the dataset:

- First there the "product_files" with product descriptions. These provide UPC codes, some description of the product, e.g. "Bud  Light, Long Neck Bottles, 6 pack, total fluid ounces" and information about the producer, vendor (in this case Anheuser Busch).

- Next there is by year, the store level dataset. This is broken down by type of store (grocery vs drug store) and some supplementary datasets for the panel dimension. This is also known as the "scanner" dataset and the main files here are labeled like this ``beer_groc_XXXX_YYYY`` where XXX and YYY match up with the weeks in each year.

- One key step is to get the UPC codes for the products that we care about, then match the store level information with the scanner data set. This will allow us to construct prices fir 

### Understanding the product file

The goal is to learn a bit about the products and eventually get the UPC's to use in the scanner price dataset.

A. For the year 2007, read in the ``prod_beer.xlsx`` file

### Companies

B. Can you find in the documentation (Hint look at Page 35 in ``Academic data set file and field description 2_3.pdf``) what column has the company name associated with each product. 

C. Find and display the top twenty companies by numbers of products. Hint: `value_counts` works well here. 

D. Do the same exercise, but using the product file for 2008-2011. Anything happen? How do the companies at the very top in 2007 compare to those in the subsequent product file. What is going on?

In [21]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [9]:
import pandas as pd

url = "D:\\BEER_DATA\\BEER_DATA\\product_files_2007\\prod_beer.xlsx"

beer = pd.read_excel(url)

In [3]:
beer["L3"].value_counts()[0:21]

ANHEUSER-BUSCH INC               1935
SABMILLER                        1061
S & P COMPANY                     571
MOLSON COORS BREWING COMPANY      562
DIAGEO                            344
HEINEKEN USA INC                  253
BOSTON BEER CO                    230
MARK ANTHONY BRANDS INC           193
LABATT USA                        182
CONSTELLATION BRANDS INC          180
HIGH FALLS BREWING                180
STAR BRAND IMPORTS                162
MATT BREWING COMPANY              121
THE GAMBRINUS COMPANY             115
MERCHANT DU VIN CORP              100
UNITED STATES BEVERAGE L.L.C.      98
PANORAMA BREWING                   89
PYRAMID BREWERIES INC              89
DISTINGUISHED BRANDS               88
BELUKUS MARKETING                  80
GREEN MOUNTAIN BEVERAGE            77
Name: L3, dtype: int64

In [6]:
url = "D:\\BEER_DATA\\BEER_DATA\\product_files_2008_2011\\prod11_beer.xlsx"

beer = pd.read_excel(url)

In [7]:
beer["L3"].value_counts()[0:21]

ANHEUSER-BUSCH INBEV           2167
SABMILLER                      1807
S & P COMPANY                   535
KPS PARTNERS                    504
DIAGEO                          443
MARK ANTHONY BRANDS INC         350
HEINEKEN USA INC                331
BOSTON BEER CO                  274
CONSTELLATION BRANDS INC        232
PAULANER HP USA                 166
SHIPYARD BREWING CO.            161
MERCHANT DU VIN CORP            153
LONG TRAIL BREWING CO.          152
CRAFT BREWERS ALLIANCE INC      150
WINERY EXCHANGE                 145
DOGFISH HEAD CRAFT BREWERY      143
THE GAMBRINUS COMPANY           127
BELLS BREWERY INC               127
NEW BELGIUM BREWING COMPANY     123
TOTAL BEVERAGE SOLUTIONS        115
MATT BREWING COMPANY            109
Name: L3, dtype: int64

### Products

For these questions **RETURN TO THE 2007 PRODUCT FILE**

B. Can you find in the documentation what column has the "brand" name associated with each product. 

C. Find and display the top ten brands by numbers of products.

In [10]:
beer["L5"].value_counts()[0:11]

BUDWEISER               207
BUD LIGHT               196
MILLER LITE             148
COORS LIGHT             136
MILLER HIGH LIFE         89
MILLER GENUINE DRAFT     84
HEINEKEN                 75
MICHELOB LIGHT           69
BUSCH                    67
SMIRNOFF TWISTED V       67
COORS                    65
Name: L5, dtype: int64

### UPCs and Product Descriptions

The want is this...we want to grab the UPC codes for several products. The products we are after will be for:
- Bud Light
- Miller Lite
- Coors Light
- Corona Extra
- Heiniken

All in the 12 packs and let's go for glass bottles. Now the issue is how to find these products in the product file. 

Here are two problems that we face:

(1) We need to figure out the size

(2) We need to figure out the format

And then select the appropriate UPCs.

#### Determining the Size of the Product.

Let's first do this for Miller Lite.

D. Create a new dataframe that only has the Miller Lite Brand. 

E. The column L9 is the UPC Description (which is a string). Can you use string methods on the dataframe (from the Chipoltle) lecture to grab the column and split the strings into a list. So each row is a list, with each item in the list being the seperate item. Can you see where the size (total number of OZs) of the product is showing up?

F. Again using string methods (hint to slice the list you do `.str(position)`) to create a new series which is only the size.

G. Use the series above to slice the dataframe so that only 144 0Z products are selected. Note 12 times 12 equals 144, hence an educated guess is that this is delivering the 12 packs we are looking for. 

#### Determining the Format of the Product.

H. Same deal as F. When looking at the description notice how you see things like "CNIBX" or "LNBTL" which correspond with the description of the format (first one is can in box, the other is long necks in box). As in F. use string methods to create a new series which details this description.

I. Slice the dataframe so that only "LNBBX", "GLBBX", "LNBTL", and "LNBCN" are selected. 

**HINT** Note the issue is we want to grab several different values. You could just to a bunch of Boolean operations. Or be smarter, so create a list of formats. Then use the [the ``.isin`` operation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html).

**Note, the issue here is that there are multiple formats which are very similar. Our strategy will be to grab them all and treat them the same. In fact, the UPCs work in the way that anytime the packaging is changed (think holiday boxes), its a new UPC***


### Scale it up...

J. Write a function that will take in the product file dataframe and the brand name (e.g. Miller Lite) and perform the operations described above (grab only 144 OZ products, grab only glass bottels) and return a new dataframe with the brands and the UPCs. 

K. Apply this function on the brands that we need. 

- BUD LIGHT
- MILLER LITE
- COORS LIGHT
- CORONA EXTRA
- HEINEKEN

In [11]:
miller_lite = beer[beer["L5"] == "MILLER LITE"]

Here is the hardes part of the assignment. There are several ways to do this. Below I create two series, one is the size and the other is the format.

In [12]:
miller_size = miller_lite.L9.str.split().str[-1] 
# This is creating a size series. So just the size, it's only one column so it is a series not a dataframe

miller_format = miller_lite.L9.str.split().str[-2]
# Same deal this is creating a series of only the format...

Then we want to select only the size and format that we want. This is where a ``boolean`` operation comes into play. The only issue is that we need to check for several formats. This is where the ``.isin`` comes in handy. It checks to see if any value in the passed (dataframe, series, list) is in the argument list. Lets see this

In [13]:
miller_lite[miller_size.isin(["144OZ"]) & miller_format.isin(["LNBBX", "GLBBX"])]

# First is the dataframe then the brackets. So within the brackets we will select stuff.

# miller_size.isin(["144OZ"]) this argument checks if each value in the series above (miller_size) is the same as
# "144OZ". Just run this command on it's own and you will see that it returns a boolean. So a bunch of true and falses
# 

# The next command miller_format.isin(["LNBBX", "GLBBX"]) checks if each value in the format series
# is either ["LNBBX", "GLBBX"]. So its asking is the value in the list ["LNBBX", "GLBBX"]. Again
# it is returning a true or false vale.

# Then we pass both miller_size.isin(["144OZ"]) & miller_format.isin(["LNBBX", "GLBBX"]) 
# Again this is just passing a bunch of true and false series into the dataframe. The \& is asking
# when are BOTH true. Which is what we want: when is it 144OZ and a bottle format.

# What does it return only 1440Z size and the correct format.


,L1,L2,L3,L4,L5,L9,Level,UPC,SY,GE,...,ITEM,*STUBSPEC 1416IS 00004,VOL_EQ,PRODUCT TYPE,TYPE OF BEER/ALE,PACKAGE,FLAVOR/SCENT,SIZE,CALORIE LEVEL,COLOR
6850,CATEGORY - BEER/ALE/ALCOHOLIC CID,DOMESTIC BEER/ALE (INC NON-ALCOH,SABMILLER,MILLER BREWING COMPANY,MILLER LITE,+MLRLT PLSNR BEER GLBBX 144OZ,9,00-01-34100-57528,0,1,...,57528,+MLRLT PLSNR BEER GLBBX 144OZ 0 1 3...,0.5,BEER,PILSNER,GLASS BOTTLE IN BOX,MISSING,MISSING,LIGHT,MISSING
6919,CATEGORY - BEER/ALE/ALCOHOLIC CID,DOMESTIC BEER/ALE (INC NON-ALCOH,SABMILLER,MILLER BREWING COMPANY,MILLER LITE,+MLRLT PLSNR BEER LNBBX 144OZ,9,00-01-00001-61295,0,1,...,61295,+MLRLT PLSNR BEER LNBBX 144OZ 0 1 ...,0.5,BEER,PILSNER,LONG NECK BTL IN BOX,MISSING,MISSING,LITE,MISSING


In [16]:
def twelve_pack_bottles(df,brand):
    
    num_oz = "144OZ" # A variable we can change if we want
    
    beer_format = ["LNBBX", "GLBBX", "LNBTL", "LNBCN"]
    # The format that we care about
    
    df_brand = df[df["L5"] == brand]
    # The brand
    
    brand_size = df_brand.L9.str.split().str[-1]
    # This is the stuff we did above

    brand_format = df_brand.L9.str.split().str[-2]
    # This is the stuff that we did above
    
    return df_brand[["L5", "UPC"]][brand_size.isin([num_oz]) & brand_format.isin(beer_format)]
    # This then returns the dataframe that we care about...

    

In [17]:
twelve_pack_bottles(beer,"BUD LIGHT")

,L5,UPC
448,BUD LIGHT,00-01-18200-00877
480,BUD LIGHT,00-04-01820-00076
481,BUD LIGHT,00-01-01820-00076
482,BUD LIGHT,00-01-18200-00769
483,BUD LIGHT,00-01-18200-00796
549,BUD LIGHT,00-01-18200-08991
550,BUD LIGHT,01-02-82000-07699
551,BUD LIGHT,01-01-82000-07699
